# Keras Model

In [81]:
import gensim
import pandas as pd
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from nltk.tokenize import TweetTokenizer

import itertools
import re
import numpy as np

# fix random seed for reproducibility
numpy.random.seed(7)

In [33]:
df = pd.read_csv('../../../Comments_FanofGame.csv')

In [60]:
df.loc[df['comment_body'].isna()]

,Unnamed: 0,comment_body,fan_of_team_playing
476957,476957,NaN,False
654089,654089,NaN,False
664634,664634,NaN,False
961154,961154,NaN,False
963550,963550,NaN,False
979510,979510,NaN,False
984894,984894,NaN,True
1089969,1089969,NaN,True
1141528,1141528,NaN,False
1145270,1145270,NaN,False


In [61]:
df.shape

(2018028, 3)

In [64]:
df.dropna().shape

(2018011, 3)

In [67]:
df_nona = df.dropna()

In [68]:
comments = df_nona.loc[:,'comment_body']
comment_list = comments.values.tolist()

In [35]:
# Borrowed some functions from the w266 utils.py file
# Miscellaneous helpers
def flatten(list_of_lists):
    """Flatten a list-of-lists into a single list."""
    return list(itertools.chain.from_iterable(list_of_lists))


# Word processing functions
def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    if not word.isupper():
        word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return constants.UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

In [36]:
%time
# tokenize
tokenizer = TweetTokenizer()
x_tokens = [tokenizer.tokenize(sentence) for sentence in comment_list if isinstance(sentence, str)]
x_tokens[0:3]

[['I',
  'love',
  'Lions',
  'fans',
  'cause',
  'we',
  'can',
  'drink',
  'bleach',
  'together',
  'this',
  'christmas',
  'eve'],
 ['I',
  'was',
  'back',
  'and',
  'forth',
  'with',
  'this',
  ',',
  'but',
  "I'm",
  'on',
  'the',
  'Fire',
  'Caldwell',
  'train',
  'now',
  '.',
  'This',
  'is',
  'embarrassing',
  '.'],
 ['Ebron', 'with', 'that', 'sick', 'Naruto', 'run', '.']]

In [42]:
x_tokens_canon = [canonicalize_word(w) for w in flatten(x_tokens)]
model_sg = gensim.models.Word2Vec(sentences=x_tokens_canon, sg=1, min_count=5, hs=1, negative=12, workers=8)

# Keras Tokenize

In [70]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(comment_list)
encoded = tokenizer.texts_to_sequences([comment_list])[0]

CPU times: user 18.5 s, sys: 60 ms, total: 18.5 s
Wall time: 18.5 s


In [71]:
%%time
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 160794
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 34.6 µs


In [ ]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)